In [1]:
import findspark

In [1]:
findspark.init()
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark import SQLContext
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import lit
from spark_sklearn import GridSearchCV

NameError: name 'findspark' is not defined

In [4]:
sc = SparkContext()
sql = SQLContext(sc)

In [8]:
df = sql.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('train.csv')

In [53]:
name = 'ps_ind_01'
udf = UserDefinedFunction(lambda x: 'Modified', StringType())
new_df = df.select(*[udf(column).alias(name) if column == name else column for column in df.columns])
new_df.select('ps_ind_01').show()

+---------+
|ps_ind_01|
+---------+
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
| Modified|
+---------+
only showing top 20 rows



In [55]:
df.select('ps_ind_01').where(df.ps_calc_15_bin == 1).show()

+---------+
|ps_ind_01|
+---------+
|        5|
|        0|
|        5|
|        1|
|        4|
|        1|
|        0|
|        4|
|        3|
|        2|
|        1|
|        0|
|        0|
|        0|
|        0|
|        2|
|        5|
|        5|
|        0|
|        0|
+---------+
only showing top 20 rows



In [32]:
df.select('ps_ind_01').show()

+---------+
|ps_ind_01|
+---------+
|        2|
|        1|
|        5|
|        0|
|        0|
|        5|
|        2|
|        5|
|        5|
|        1|
|        5|
|        2|
|        2|
|        1|
|        5|
|        5|
|        1|
|        5|
|        5|
|        0|
+---------+
only showing top 20 rows



In [34]:
df.select('ps_ind_01').distinct().show()

+---------+
|ps_ind_01|
+---------+
|        1|
|        6|
|        3|
|        5|
|        4|
|        7|
|        2|
|        0|
+---------+



In [36]:
df.select('ps_ind_01').count()

595212

In [47]:
df.groupby('ps_ind_04_cat').agg({'ps_calc_11': "avg", 'ps_car_09_cat': "max"}).show()

+-------------+-----------------+------------------+
|ps_ind_04_cat|  avg(ps_calc_11)|max(ps_car_09_cat)|
+-------------+-----------------+------------------+
|           -1|5.506024096385542|                 3|
|            1| 5.43588111087829|                 4|
|            0|5.445301399276584|                 4|
+-------------+-----------------+------------------+



In [52]:
df.withColumn("x4", lit(0)).select('x4').show()

+---+
| x4|
+---+
|  0|
|  0|
|  0|
|  0|
|  0|
|  0|
|  0|
|  0|
|  0|
|  0|
|  0|
|  0|
|  0|
|  0|
|  0|
|  0|
|  0|
|  0|
|  0|
|  0|
+---+
only showing top 20 rows



In [22]:
assembler = VectorAssembler(
    inputCols=df.schema.names,
    outputCol="features")

In [23]:
output = assembler.transform(df)

In [28]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr.setLabelCol("target")
lr.fit(output)

LogisticRegression_498ab587ae77412f05b0